<a href="https://colab.research.google.com/github/yanna-torres/CKP9011-ciencia-de-dados/blob/lista-02/lista02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

 # Lista 02 - Tratamento de dados
        
> Yanna Torres Gonçalves
>
> Mestrado em Ciências da Computação
>
> Matrícula: 587299

In [1]:
!pip install duckdb

In [2]:
import pandas as pd
import numpy as np
import re
import duckdb

import string
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

## a) Ler o dataset fakeTelegram.BR_2022.csv

Utilizamos o mesmo dataset da lista passada, acessando pelo mesmo link.

[https://github.com/yanna-torres/CKP9011-ciencia-de-dados/blob/lista-01/data/fakeTelegram.BR_2022.csv](https://github.com/yanna-torres/CKP9011-ciencia-de-dados/blob/lista-01/data/fakeTelegram.BR_2022.csv)

In [3]:
data_url = "https://media.githubusercontent.com/media/yanna-torres/CKP9011-ciencia-de-dados/refs/heads/lista-01/data/fakeTelegram.BR_2022.csv"

df = pd.read_csv(data_url)

In [4]:
df.head(10)

,date_message,id_member_anonymous,id_group_anonymous,media,media_type,media_url,has_media,has_media_url,trava_zap,text_content_anonymous,dataset_info_id,date_system,score_sentiment,score_misinformation,id_message,message_type,messenger,media_name,media_md5
0,2022-10-05 06:25:04,1078cc958f0febe28f4d03207660715f,12283e08a2eb5789201e105b34489ee7,NaN,NaN,NaN,False,False,False,Então é Fato Renato o áudio que eu ouvi no wha...,5,2022-10-05 06:25:28.863641,0.0000,NaN,16385,Texto,telegram,NaN,NaN
1,2022-10-05 06:25:08,NaN,12283e08a2eb5789201e105b34489ee7,NaN,NaN,NaN,False,False,False,"Saiu no YouTube do presidente a 8 horas atrás,...",5,2022-10-05 06:25:28.926311,0.0644,NaN,16386,Texto,telegram,NaN,NaN
2,2022-10-05 06:26:28,92a2d8fd7144074f659d1d29dc3751da,9f2d7394334eb224c061c9740b5748fc,NaN,NaN,NaN,False,False,False,"É isso, nossa parte já foi quase toda feita. N...",5,2022-10-05 06:26:29.361949,-0.3551,0.157242,16366,Texto,telegram,NaN,NaN
3,2022-10-05 06:27:28,d60aa38f62b4977426b70944af4aff72,c8f2de56550ed0bf85249608b7ead93d,94dca4cda503100ebfda7ce2bcc060eb.jpg,image/jpg,NaN,True,False,False,GENTE ACHEI ELES EM UMA SEITA MAÇONÁRICA,5,2022-10-05 06:27:29.935624,0.0000,NaN,19281,Imagem,telegram,NaN,94dca4cda503100ebfda7ce2bcc060eb
4,2022-10-05 06:27:44,cd6979b0b5265f08468fa1689b6300ce,e56ec342fc599ebb4ed89655eb6f03aa,5ad5c8bbe9da93a37fecf3e5aa5b0637.jpg,image/jpg,NaN,True,False,False,NaN,5,2022-10-05 06:28:29.316325,NaN,NaN,507185,Imagem,telegram,NaN,5ad5c8bbe9da93a37fecf3e5aa5b0637
5,2022-10-05 06:28:30,NaN,b52442a5fbc459ae590dca0d215e32f9,NaN,NaN,NaN,False,False,False,Kķkkkkk to rindo até agora....Quem disse q ia ...,5,2022-10-05 06:29:29.046694,0.7003,0.197813,2735,Texto,telegram,NaN,NaN
6,2022-10-05 06:29:09,3b685d44ff197b98d7c9e99b8f6b5281,b52442a5fbc459ae590dca0d215e32f9,NaN,NaN,NaN,False,False,False,*SE ALGUÉM TE PERGUNTAR O QUE FOI QUE BOLSONAR...,5,2022-10-05 06:29:29.33448,0.9716,0.974258,2736,Texto,telegram,NaN,NaN
7,2022-10-05 06:43:51,a7db4ff6a6d35e57be5bcf02f400cec6,857cd5311da1bdc15eb9e6918a47c6c6,4fe93a3772445f64173ef03db1fed83c.jpg,image/jpg,NaN,True,False,False,NaN,5,2022-10-05 06:43:54.51408,NaN,NaN,891535,Imagem,telegram,NaN,4fe93a3772445f64173ef03db1fed83c
8,2022-10-05 07:09:39,d2c33afd13a3165be031b6c244f7140d,d9c080b9db8cd9d0e88e870ce782c01a,30f29cca618753961cb97be95e87e3b4.jpg,image/jpg,NaN,True,False,False,NaN,5,2022-10-05 07:10:16.569761,NaN,NaN,314861,Imagem,telegram,NaN,30f29cca618753961cb97be95e87e3b4
9,2022-10-05 06:29:48,a7e85072244cae15446c9d517dc01a1a,b8a8737812c7fd7d3e0bdbb65ef6306f,NaN,NaN,www.marketingdigitalparavencer.com.br,False,True,False,O Deputado Federal pelo NOVO e que foi candida...,5,2022-10-05 06:29:49.901419,-0.8779,0.035876,7248,Texto,telegram,NaN,NaN


In [5]:
df.describe()

,dataset_info_id,score_sentiment,score_misinformation,id_message
count,557586.0,444157.000000,167238.000000,5.575860e+05
mean,5.0,0.017330,0.312245,4.450617e+05
std,0.0,0.464165,0.293699,4.860211e+05
min,5.0,-1.000000,0.000003,2.000000e+00
25%,5.0,-0.177900,0.078454,2.127500e+04
50%,5.0,0.000000,0.197577,1.210935e+05
75%,5.0,0.318200,0.490351,9.726045e+05
max,5.0,0.999200,1.000000,1.516436e+06


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 557586 entries, 0 to 557585
Data columns (total 19 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   date_message            557586 non-null  object 
 1   id_member_anonymous     234245 non-null  object 
 2   id_group_anonymous      557586 non-null  object 
 3   media                   332605 non-null  object 
 4   media_type              332605 non-null  object 
 5   media_url               157445 non-null  object 
 6   has_media               557586 non-null  bool   
 7   has_media_url           557586 non-null  bool   
 8   trava_zap               557586 non-null  bool   
 9   text_content_anonymous  444201 non-null  object 
 10  dataset_info_id         557586 non-null  int64  
 11  date_system             557586 non-null  object 
 12  score_sentiment         444157 non-null  float64
 13  score_misinformation    167238 non-null  float64
 14  id_message          

## b) Remova os trava-zaps

Utilizamos a mesma função da lista 01 criada para validar os dados da coluna `trava_zap`. Após atualizar a coluna, criamos um novo dataframe com os dados sem trava-zaps.

In [7]:
def has_trava_zap(message):
  if not isinstance(message, str):
    return False  # Garante que só analisa strings

  # Regra 1: muito longo
  if len(message) > 10000:
    return True

  # Regra 2: caracteres invisíveis
  invisible = ''.join(chr(c) for c in range(0x200B, 0x200F + 1)) + '\uFEFF'
  if sum(message.count(c) for c in invisible) > 100:
    return True

  # Regra 3: emojis repetidos
  emoji_pattern = re.compile("[\U0001F600-\U0001F64F\U0001F300-\U0001F5FF"
                              "\U0001F680-\U0001F6FF\U0001F1E0-\U0001F1FF]+", flags=re.UNICODE)
  emojis = emoji_pattern.findall(message)
  if any(len(e) > 20 for e in emojis):
    return True

  # Regra 4: baixa diversidade
  unique = set(message)
  if len(unique) < 10 and len(message) > 500:
    return True

  return False

In [8]:
df['trava_zap'] = df['text_content_anonymous'].apply(has_trava_zap)

In [9]:
df_clean = df[df['trava_zap'] == False]

In [10]:
df_clean.drop(columns=['trava_zap'], inplace=True)

<ipython-input-10-d18d2c2b7bb2>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean.drop(columns=['trava_zap'], inplace=True)


In [11]:
df_clean.head(10)

,date_message,id_member_anonymous,id_group_anonymous,media,media_type,media_url,has_media,has_media_url,text_content_anonymous,dataset_info_id,date_system,score_sentiment,score_misinformation,id_message,message_type,messenger,media_name,media_md5
0,2022-10-05 06:25:04,1078cc958f0febe28f4d03207660715f,12283e08a2eb5789201e105b34489ee7,NaN,NaN,NaN,False,False,Então é Fato Renato o áudio que eu ouvi no wha...,5,2022-10-05 06:25:28.863641,0.0000,NaN,16385,Texto,telegram,NaN,NaN
1,2022-10-05 06:25:08,NaN,12283e08a2eb5789201e105b34489ee7,NaN,NaN,NaN,False,False,"Saiu no YouTube do presidente a 8 horas atrás,...",5,2022-10-05 06:25:28.926311,0.0644,NaN,16386,Texto,telegram,NaN,NaN
2,2022-10-05 06:26:28,92a2d8fd7144074f659d1d29dc3751da,9f2d7394334eb224c061c9740b5748fc,NaN,NaN,NaN,False,False,"É isso, nossa parte já foi quase toda feita. N...",5,2022-10-05 06:26:29.361949,-0.3551,0.157242,16366,Texto,telegram,NaN,NaN
3,2022-10-05 06:27:28,d60aa38f62b4977426b70944af4aff72,c8f2de56550ed0bf85249608b7ead93d,94dca4cda503100ebfda7ce2bcc060eb.jpg,image/jpg,NaN,True,False,GENTE ACHEI ELES EM UMA SEITA MAÇONÁRICA,5,2022-10-05 06:27:29.935624,0.0000,NaN,19281,Imagem,telegram,NaN,94dca4cda503100ebfda7ce2bcc060eb
4,2022-10-05 06:27:44,cd6979b0b5265f08468fa1689b6300ce,e56ec342fc599ebb4ed89655eb6f03aa,5ad5c8bbe9da93a37fecf3e5aa5b0637.jpg,image/jpg,NaN,True,False,NaN,5,2022-10-05 06:28:29.316325,NaN,NaN,507185,Imagem,telegram,NaN,5ad5c8bbe9da93a37fecf3e5aa5b0637
5,2022-10-05 06:28:30,NaN,b52442a5fbc459ae590dca0d215e32f9,NaN,NaN,NaN,False,False,Kķkkkkk to rindo até agora....Quem disse q ia ...,5,2022-10-05 06:29:29.046694,0.7003,0.197813,2735,Texto,telegram,NaN,NaN
6,2022-10-05 06:29:09,3b685d44ff197b98d7c9e99b8f6b5281,b52442a5fbc459ae590dca0d215e32f9,NaN,NaN,NaN,False,False,*SE ALGUÉM TE PERGUNTAR O QUE FOI QUE BOLSONAR...,5,2022-10-05 06:29:29.33448,0.9716,0.974258,2736,Texto,telegram,NaN,NaN
7,2022-10-05 06:43:51,a7db4ff6a6d35e57be5bcf02f400cec6,857cd5311da1bdc15eb9e6918a47c6c6,4fe93a3772445f64173ef03db1fed83c.jpg,image/jpg,NaN,True,False,NaN,5,2022-10-05 06:43:54.51408,NaN,NaN,891535,Imagem,telegram,NaN,4fe93a3772445f64173ef03db1fed83c
8,2022-10-05 07:09:39,d2c33afd13a3165be031b6c244f7140d,d9c080b9db8cd9d0e88e870ce782c01a,30f29cca618753961cb97be95e87e3b4.jpg,image/jpg,NaN,True,False,NaN,5,2022-10-05 07:10:16.569761,NaN,NaN,314861,Imagem,telegram,NaN,30f29cca618753961cb97be95e87e3b4
9,2022-10-05 06:29:48,a7e85072244cae15446c9d517dc01a1a,b8a8737812c7fd7d3e0bdbb65ef6306f,NaN,NaN,www.marketingdigitalparavencer.com.br,False,True,O Deputado Federal pelo NOVO e que foi candida...,5,2022-10-05 06:29:49.901419,-0.8779,0.035876,7248,Texto,telegram,NaN,NaN


In [12]:
df_clean.describe()

,dataset_info_id,score_sentiment,score_misinformation,id_message
count,556305.0,442876.000000,166416.000000,5.563050e+05
mean,5.0,0.017676,0.311496,4.448514e+05
std,0.0,0.463547,0.293146,4.859544e+05
min,5.0,-1.000000,0.000003,2.000000e+00
25%,5.0,-0.177900,0.078334,2.125100e+04
50%,5.0,0.000000,0.196892,1.208690e+05
75%,5.0,0.318200,0.488192,9.724220e+05
max,5.0,0.999200,1.000000,1.516436e+06


In [13]:
df_clean.info()

<class 'pandas.core.frame.DataFrame'>
Index: 556305 entries, 0 to 557585
Data columns (total 18 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   date_message            556305 non-null  object 
 1   id_member_anonymous     233738 non-null  object 
 2   id_group_anonymous      556305 non-null  object 
 3   media                   332261 non-null  object 
 4   media_type              332261 non-null  object 
 5   media_url               157196 non-null  object 
 6   has_media               556305 non-null  bool   
 7   has_media_url           556305 non-null  bool   
 8   text_content_anonymous  442920 non-null  object 
 9   dataset_info_id         556305 non-null  int64  
 10  date_system             556305 non-null  object 
 11  score_sentiment         442876 non-null  float64
 12  score_misinformation    166416 non-null  float64
 13  id_message              556305 non-null  int64  
 14  message_type            5

---

Adicionando uma coluna do texto sem stop words para facilitar itens futuros.

In [14]:
stop_words = set(stopwords.words('portuguese'))

def clean_text(text):
  if not isinstance(text, str):
    return ''
  text_no_punct = text.translate(str.maketrans('', '', string.punctuation))
  tokens = word_tokenize(text_no_punct)
  filtered = [word for word in tokens if word.lower() not in stop_words]
  return ' '.join(filtered)

df_clean['text_no_stopwords'] = df_clean['text_content_anonymous'].apply(clean_text)

<ipython-input-14-6c87d28faaa3>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean['text_no_stopwords'] = df_clean['text_content_anonymous'].apply(clean_text)


---

## c) Exportar os dados para um arquivo Parquet

Para exportar os dados para um arquivo Parquet, utilizamos a função `to_parquet` do Pandas.

In [15]:
df_clean.to_parquet("fakeTelegramBR_2022_clean.parquet")

## d) Exportar os dados para o DuckDB

Para exportar os dados para um banco do DuckDB basta utilizar o comando SQL:

```sql
CREATE TABLE nome_da_table AS SELECT * FROM nome_do_dataframe
```

[Documentação DuckDB](https://duckdb.org/docs/stable/guides/python/import_pandas.html)

In [16]:
conn = duckdb.connect()

conn.execute("CREATE TABLE messages AS SELECT * FROM df_clean")

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [17]:
result = conn.execute("SELECT * FROM messages LIMIT 10").df()

In [18]:
result

,date_message,id_member_anonymous,id_group_anonymous,media,media_type,media_url,has_media,has_media_url,text_content_anonymous,dataset_info_id,date_system,score_sentiment,score_misinformation,id_message,message_type,messenger,media_name,media_md5,text_no_stopwords
0,2022-10-05 06:25:04,1078cc958f0febe28f4d03207660715f,12283e08a2eb5789201e105b34489ee7,None,None,None,False,False,Então é Fato Renato o áudio que eu ouvi no wha...,5,2022-10-05 06:25:28.863641,0.0000,NaN,16385,Texto,telegram,None,None,Então Fato Renato áudio ouvi whatsapp ocorreu ...
1,2022-10-05 06:25:08,None,12283e08a2eb5789201e105b34489ee7,None,None,None,False,False,"Saiu no YouTube do presidente a 8 horas atrás,...",5,2022-10-05 06:25:28.926311,0.0644,NaN,16386,Texto,telegram,None,None,Saiu YouTube presidente 8 horas atrás infelizm...
2,2022-10-05 06:26:28,92a2d8fd7144074f659d1d29dc3751da,9f2d7394334eb224c061c9740b5748fc,None,None,None,False,False,"É isso, nossa parte já foi quase toda feita. N...",5,2022-10-05 06:26:29.361949,-0.3551,0.157242,16366,Texto,telegram,None,None,parte quase toda feita segundo turno completam...
3,2022-10-05 06:27:28,d60aa38f62b4977426b70944af4aff72,c8f2de56550ed0bf85249608b7ead93d,94dca4cda503100ebfda7ce2bcc060eb.jpg,image/jpg,None,True,False,GENTE ACHEI ELES EM UMA SEITA MAÇONÁRICA,5,2022-10-05 06:27:29.935624,0.0000,NaN,19281,Imagem,telegram,None,94dca4cda503100ebfda7ce2bcc060eb,GENTE ACHEI SEITA MAÇONÁRICA
4,2022-10-05 06:27:44,cd6979b0b5265f08468fa1689b6300ce,e56ec342fc599ebb4ed89655eb6f03aa,5ad5c8bbe9da93a37fecf3e5aa5b0637.jpg,image/jpg,None,True,False,None,5,2022-10-05 06:28:29.316325,NaN,NaN,507185,Imagem,telegram,None,5ad5c8bbe9da93a37fecf3e5aa5b0637,
5,2022-10-05 06:28:30,None,b52442a5fbc459ae590dca0d215e32f9,None,None,None,False,False,Kķkkkkk to rindo até agora....Quem disse q ia ...,5,2022-10-05 06:29:29.046694,0.7003,0.197813,2735,Texto,telegram,None,None,Kķkkkkk to rindo agoraQuem disse q ia fazer ac...
6,2022-10-05 06:29:09,3b685d44ff197b98d7c9e99b8f6b5281,b52442a5fbc459ae590dca0d215e32f9,None,None,None,False,False,*SE ALGUÉM TE PERGUNTAR O QUE FOI QUE BOLSONAR...,5,2022-10-05 06:29:29.33448,0.9716,0.974258,2736,Texto,telegram,None,None,ALGUÉM PERGUNTAR BOLSONARO FEZ BOM BRASIL MOST...
7,2022-10-05 06:43:51,a7db4ff6a6d35e57be5bcf02f400cec6,857cd5311da1bdc15eb9e6918a47c6c6,4fe93a3772445f64173ef03db1fed83c.jpg,image/jpg,None,True,False,None,5,2022-10-05 06:43:54.51408,NaN,NaN,891535,Imagem,telegram,None,4fe93a3772445f64173ef03db1fed83c,
8,2022-10-05 07:09:39,d2c33afd13a3165be031b6c244f7140d,d9c080b9db8cd9d0e88e870ce782c01a,30f29cca618753961cb97be95e87e3b4.jpg,image/jpg,None,True,False,None,5,2022-10-05 07:10:16.569761,NaN,NaN,314861,Imagem,telegram,None,30f29cca618753961cb97be95e87e3b4,
9,2022-10-05 06:29:48,a7e85072244cae15446c9d517dc01a1a,b8a8737812c7fd7d3e0bdbb65ef6306f,None,None,www.marketingdigitalparavencer.com.br,False,True,O Deputado Federal pelo NOVO e que foi candida...,5,2022-10-05 06:29:49.901419,-0.8779,0.035876,7248,Texto,telegram,None,None,Deputado Federal NOVO candidato Governo RJ Pau...


## e) Utlizando o DuckDB recupere

### 1. A quantidade de mensagens

In [19]:
quant_messages = conn.execute("SELECT COUNT(*) AS total_messages FROM messages").df()
quant_messages

,total_messages
0,556305


### 2. A quantidade de usuários

In [20]:
quant_users = conn.execute("SELECT COUNT(DISTINCT id_member_anonymous) AS total_users FROM messages").df()
quant_users

,total_users
0,14786


### 3. A quantidade de grupos

In [21]:
quant_groups = conn.execute("SELECT COUNT(DISTINCT id_group_anonymous) AS total_groups FROM messages").df()
quant_groups

,total_groups
0,178


### 4. Quantidade de mensagens que possuem apenas texto

In [22]:
quant_messages_only_text = conn.execute("SELECT COUNT(*) AS total_messages FROM messages WHERE text_content_anonymous IS NOT NULL AND has_media = False").df()
quant_messages_only_text

,total_messages
0,224029


### 5. Quantidade de mensagens contendo mídias

In [23]:
quant_messages_with_media = conn.execute("SELECT COUNT(*) AS total_messages FROM messages WHERE has_media = True").df()
quant_messages_with_media

,total_messages
0,332261


### 6. Quantidade de mensagens por tipo de mídia (jpg, mp4 etc)

In [24]:
quant_messages_per_media_type_df = conn.execute("""
    SELECT media_type, COUNT(*) AS total_messages
    FROM messages
    WHERE has_media = True
    GROUP BY media_type
""").df()
quant_messages_per_media_type_df

,media_type,total_messages
0,url,100781
1,image/pdf,230
2,application/pdf,2850
3,application/x-partial-download,1
4,image/apk,234
...,...,...
62,image/02d0e99ce96591d1e3d354ebb0690607,2
63,application/x-msi,1
64,image/08e8201b6f32c358808c7ac89d8c8422,1
65,video/x-flv,6


In [25]:
quant_messages_per_media_type_df.sort_values(by='total_messages', ascending=False).head(10)

,media_type,total_messages
29,image/jpg,200202
0,url,100781
30,video/mp4,18469
32,application/vnd.android.package-archive,7159
2,application/pdf,2850
31,image/mp4,1589
4,image/apk,234
1,image/pdf,230
11,image/f4v,83
6,video/mpeg,69


### 7. Quantidade de mensagens por estado

### 8. Quantidade de usuários por estado

### 9. Relação quantidade de usuários por quantidade de mensagens por estado

### 10. Quantidade de mensagens por país

### 11. Quantidade de mensagens Brasil X Países Estrangeiros

O dataset não possui informações de telefone, de identificação do estado ou de país de origem de cada mensagem.

In [26]:
columns = conn.execute("SELECT * FROM messages LIMIT 0").df().columns
columns

Index(['date_message', 'id_member_anonymous', 'id_group_anonymous', 'media',
       'media_type', 'media_url', 'has_media', 'has_media_url',
       'text_content_anonymous', 'dataset_info_id', 'date_system',
       'score_sentiment', 'score_misinformation', 'id_message', 'message_type',
       'messenger', 'media_name', 'media_md5', 'text_no_stopwords'],
      dtype='object')

### 12. As 30 URLs que mais se repetem (mais compartilhadas)

In [27]:
top_urls = conn.execute("""
    SELECT media_url as url, COUNT(*) AS total
    FROM messages
    WHERE url IS NOT NULL
    GROUP BY url
    ORDER BY total DESC
    LIMIT 30
""").df()
top_urls

,url,total
0,t.me/alexeconomia,4159
1,https://youtube.com/c/especulandoosfatosoficial,1607
2,t.me/fimtaproximo,1581
3,t.me/+EWlGMatRZGg3OTlh,1419
4,https://youtu.be/qbTzhB0akt8,1159
5,https://youtu.be/zDuOoyhyN-4,1015
6,t.me/+ewZIPdZ42vEyNzJh,772
7,https://t.me/canalselvabrasiloficial,709
8,https://youtu.be/4DHk9KZ01HM,653
9,T.me/Arthur_Senna_Trader2,640


### 13. Os 30 domínios que mais se repetem (mais compartilhados)

In [28]:
regex_domains = "LOWER(regexp_extract(regexp_replace(media_url, 'https?://(www\.)?', ''), '^([^/]+)', 1))"
top_domains = conn.execute(f"""
    SELECT
        {regex_domains} AS domain,
        COUNT(*) AS total
    FROM messages
    WHERE media_url IS NOT NULL
    GROUP BY domain
    ORDER BY total DESC
    LIMIT 30
""").df()
top_domains

,domain,total
0,youtu.be,52801
1,t.me,21290
2,youtube.com,14754
3,terrabrasilnoticias.com,8277
4,instagram.com,6649
5,jornaldacidadeonline.com.br,4812
6,twitter.com,4437
7,gazetabrasil.com.br,1781
8,pleno.news,1602
9,facebook.com,1364


### 14. Os 30 usuários mais ativos

In [29]:
top_users = conn.execute("""
    SELECT id_member_anonymous, COUNT(*) AS total
    FROM messages
    WHERE id_member_anonymous IS NOT NULL
    GROUP BY id_member_anonymous
    ORDER BY total DESC
    LIMIT 30
""").df()
top_users

,id_member_anonymous,total
0,abe534d581ec6d552243d6955d3c3cd8,12289
1,1665e22b0f564cd46d343f7677014821,5452
2,1ac091b8ed5c4e42383f1b4ff4cc9b2d,5035
3,c743967449a387ad2c1c7e03b2c45b36,3019
4,e7998863ac2a40086657fab4a6b463c9,1928
5,e8fd8fee8c39342d37993775da7756d5,1706
6,2f4be6244ede15b46e8329a2c975be30,1616
7,d49f81df0c75d1d72bee6c5b2d707da0,1571
8,4f7d493f0f6222d56e5b19a4f7c336cc,1446
9,773b9bd5b02a2e96f9d732c29bfcb663,1411


### 15. Os 30 usuários que mais compartilharam texto

In [30]:
top_text_users = conn.execute("""
    SELECT id_member_anonymous, COUNT(*) AS total
    FROM messages
    WHERE text_content_anonymous IS NOT NULL AND text_content_anonymous != '' AND id_member_anonymous IS NOT NULL
    GROUP BY id_member_anonymous
    ORDER BY total DESC
    LIMIT 30
""").df()
top_text_users

,id_member_anonymous,total
0,abe534d581ec6d552243d6955d3c3cd8,12289
1,1665e22b0f564cd46d343f7677014821,5452
2,1ac091b8ed5c4e42383f1b4ff4cc9b2d,3997
3,c743967449a387ad2c1c7e03b2c45b36,3019
4,d49f81df0c75d1d72bee6c5b2d707da0,1571
5,e7998863ac2a40086657fab4a6b463c9,1401
6,4f7d493f0f6222d56e5b19a4f7c336cc,1364
7,773b9bd5b02a2e96f9d732c29bfcb663,1313
8,e8fd8fee8c39342d37993775da7756d5,1225
9,f233cf8b1d4ede06f32199a7e0081bf5,1146


### 16. Os 30 usuários que mais compartilharam mídias

In [31]:
top_media_users = conn.execute("""
    SELECT id_member_anonymous, COUNT(*) AS total
    FROM messages
    WHERE (has_media = TRUE OR (media_url IS NOT NULL AND media_url != ''))
    AND id_member_anonymous IS NOT NULL
    GROUP BY id_member_anonymous
    ORDER BY total DESC
    LIMIT 30
""").df()
top_media_users

,id_member_anonymous,total
0,1ac091b8ed5c4e42383f1b4ff4cc9b2d,4659
1,1665e22b0f564cd46d343f7677014821,3329
2,e7998863ac2a40086657fab4a6b463c9,1527
3,2f4be6244ede15b46e8329a2c975be30,1507
4,d49f81df0c75d1d72bee6c5b2d707da0,1500
5,4f7d493f0f6222d56e5b19a4f7c336cc,1405
6,c052c859b42c5a1923c22f5a201de746,1315
7,773b9bd5b02a2e96f9d732c29bfcb663,1173
8,56b8359fd127312651b80b8ed8030085,1153
9,e8fd8fee8c39342d37993775da7756d5,1012


### 17. As 30 mensagens mais compartilhadas

In [32]:
top_shared_messages = conn.execute("""
    SELECT text_content_anonymous as message, COUNT(*) AS total_shares
    FROM messages WHERE message IS NOT NULL OR message != ''
    GROUP BY message
    ORDER BY total_shares DESC
    LIMIT 30
""").df()
top_shared_messages

,message,total_shares
0,This community was blocked in Brazil following...,17422
1,Rough_sex🙈,1134
2,Anal sex🙈,1118
3,سکس مردان ازبک با زن انگلیسی با این vpn از سای...,1019
4,https://youtu.be/qbTzhB0akt8,758
5,https://youtu.be/zDuOoyhyN-4,712
6,فیلم سوپر با زیرنویس فارسی ببین😍😍\nبا این فیلت...,632
7,Rough😱,570
8,Foto de Nélia Barros,548
9,We had no choice but to remain in the shadows....,480


### 18. As 30 mensagens mais compartilhadas em grupos diferentes

In [33]:
top_shared_messages_in_groups = conn.execute("""
    SELECT text_content_anonymous as message, COUNT(DISTINCT id_group_anonymous) AS total_groups
    FROM messages
    WHERE message IS NOT NULL AND id_group_anonymous IS NOT NULL AND message != ''
    GROUP BY message
    ORDER BY total_groups DESC
    LIMIT 30
""").df()
top_shared_messages_in_groups

,message,total_groups
0,This community was blocked in Brazil following...,59
1,BOA NOITE A TODOS. O QUE EU VOU DIZER É SÉRIO ...,36
2,Hoje às 18 horas no canal Inteligência Ltda do...,35
3,[USER],32
4,https://youtu.be/zDuOoyhyN-4,31
5,https://youtu.be/qbTzhB0akt8,30
6,Ajudem a subir a #JanonesAmigoDePedofilo\n\nPo...,29
7,*ATENÇÃO* \n🚨🚨🚨🚨🚨🚨🚨🚨🚨\n\n*CHAMADA URGENTE* Pat...,29
8,"DE NADA IRÁ ADIANTAR FISCALIZAR AS URNAS, SE N...",29
9,OS PTISTAS NÃO QUER QUE AGENTE MOSTRE QUE VOTO...,29


### 19. Mensagens idênticas compartilhadas pelo mesmo usuário (e suas quantidades)

In [34]:
repeated_messages_by_user = conn.execute("""
    SELECT
        id_member_anonymous,
        text_content_anonymous,
        COUNT(*) AS total
    FROM messages
    WHERE text_content_anonymous IS NOT NULL AND text_content_anonymous != ''
          AND id_member_anonymous IS NOT NULL
    GROUP BY id_member_anonymous, text_content_anonymous
    HAVING COUNT(*) > 1
    ORDER BY total DESC
""").df()
repeated_messages_by_user

,id_member_anonymous,text_content_anonymous,total
0,f233cf8b1d4ede06f32199a7e0081bf5,This community was blocked in Brazil following...,838
1,c743967449a387ad2c1c7e03b2c45b36,This community was blocked in Brazil following...,532
2,7696d5103cdb8ac352d748a1db1126b0,This community was blocked in Brazil following...,530
3,1665e22b0f564cd46d343f7677014821,Bem vindo(a) ao grupo Ipirá Notícias. \n\nComp...,357
4,a398999c55f8d6f0c65760522ae12e45,This community was blocked in Brazil following...,308
...,...,...,...
18706,8316e0fd2ced47ecd8b348371bee6ec7,This community was blocked in Brazil following...,2
18707,fd1e71062efb872a98faa4463503af24,⚠️ FURADA... CUIDADO! ⚠️\n\n❌ ISSO NÃO ESTÁ CO...,2
18708,048fad9af3d2c9c92e04cfb0f0231e29,https://youtube.com/shorts/cqykZ2KwWXE?feature...,2
18709,67d8d99d20ae674d3f5aab7cbad28d90,https://www.instagram.com/reel/CkzljsagWBo/?ig...,2


### 20. Mensagens idênticas compartilhadas pelo mesmo usuário em grupos distintos (e suas quantidades)

In [35]:
repeated_messages_by_user_groups = conn.execute("""
    SELECT
        id_member_anonymous,
        text_content_anonymous as message,
        COUNT(DISTINCT id_group_anonymous) AS total_groups
    FROM messages
    WHERE text_content_anonymous IS NOT NULL
          AND message != ''
          AND id_member_anonymous IS NOT NULL
          AND id_group_anonymous IS NOT NULL
    GROUP BY id_member_anonymous, message
    HAVING COUNT(DISTINCT id_group_anonymous) > 1
    ORDER BY total_groups DESC
""").df()
repeated_messages_by_user_groups

,id_member_anonymous,message,total_groups
0,a7840e7844020149e197272748965862,ALERTA\nESSE GRUPO FOI CRIADO PELA ESQUERDA CO...,16
1,326d0a2f1bc5a1ede446fcf5dc31ff2e,💣💥((((( URGENTÍSSIMO )))))💥💣\nESSE GRUPO É UMA...,14
2,326d0a2f1bc5a1ede446fcf5dc31ff2e,💣💣💣💥💥🔰BOMBA BOMBA🔰💥💥💣💣💣\nVCS ESTÃO ACHANDO QUE...,12
3,29dda76df3384c28a07e8bad1ee9ceed,BOA NOITE A TODOS. O QUE EU VOU DIZER É SÉRIO ...,11
4,dc60c84ed8dec03b667c577e124c5540,INTERCESSÃO PELO BRASIL!!\n\nChegou a hora da ...,10
...,...,...,...
13160,065039508ed6e276e5f092dd22eaef94,Barreiras Bahia\nCidade com 158 mil hab comput...,2
13161,d9acc4da8bfedac7f969934f048434eb,*Esse é EXEMPLO de aberrações que são colocada...,2
13162,27f5833380581fab3453c1acd26427d5,"A cigarra, com raiva da formiga, votou no inse...",2
13163,4d6b2ceaf66bcee86d32cbe5e01eac08,"Pode chegar a pilotar o jato, mas terá que se ...",2


### 21. Os 30 unigramas, bigramas e trigramas mais compartilhados

In [53]:
top_30_1gram = conn.execute("""
SELECT
    word AS ngram,
    COUNT(*) AS total
FROM (
    SELECT UNNEST(STRING_SPLIT(text_no_stopwords, ' ')) AS word
    FROM messages
    WHERE text_no_stopwords IS NOT NULL AND text_no_stopwords != ''
)
GROUP BY word
ORDER BY total DESC
LIMIT 30;
""").df()
top_30_1gram

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,ngram,total
0,USER,63079
1,Bolsonaro,49423
2,13,46394
3,—,36114
4,Brasil,35327
5,Lula,32033
6,the,29561
7,TSE,28461
8,todos,24830
9,of,24280


In [51]:
query = """
WITH tokens AS (
    SELECT word
    FROM (
        SELECT unnest(string_split_regex(lower(text_content_anonymous), '[^\\p{L}0-9]+')) as word
        FROM messages
        WHERE text_content_anonymous IS NOT NULL AND text_content_anonymous != ''
    )
    WHERE word NOT LIKE 'http%' AND word NOT LIKE 'www%' AND word NOT LIKE 'com%' AND word NOT LIKE 't.me%' AND word NOT LIKE 'youtu%' AND length(word) > 1
),
ngrams AS (
    SELECT
        word || ' ' || lead(word, 1) OVER () as two_gram
    FROM tokens
)
SELECT
    two_gram as "2gram",
    count(*) as count
FROM ngrams
WHERE two_gram NOT LIKE '% % %'
GROUP BY two_gram
ORDER BY count DESC
LIMIT 30;
"""
top_2grams = conn.execute(query).df()
top_2grams

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,2gram,count
0,13 13,42681
1,of the,18472
2,in brazil,17499
3,this was,17445
4,electoral court,17428
5,court tse,17426
6,superior electoral,17426
7,the superior,17424
8,decision of,17423
9,brazil following,17422


In [52]:
query = """
WITH tokens AS (
    SELECT word
    FROM (
        SELECT unnest(string_split_regex(lower(text_content_anonymous), '[^\\p{L}0-9]+')) AS word
        FROM messages
        WHERE text_content_anonymous IS NOT NULL AND text_content_anonymous != ''
    )
    WHERE word NOT LIKE 'http%'
      AND word NOT LIKE 'www%'
      AND word NOT LIKE 'com%'
      AND word NOT LIKE 't.me%'
      AND word NOT LIKE 'youtu%'
      AND length(word) > 1
),
ngrams AS (
    SELECT
        word || ' ' || lead(word, 1) OVER () || ' ' || lead(word, 2) OVER () AS three_gram
    FROM tokens
)
SELECT
    three_gram AS "3gram",
    count(*) AS count
FROM ngrams
WHERE three_gram NOT LIKE '% % % %'
GROUP BY three_gram
ORDER BY count DESC
LIMIT 30;
"""
top_3grams = conn.execute(query).df()
top_3grams

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,3gram,count
0,13 13 13,41951
1,the superior electoral,17424
2,superior electoral court,17424
3,brazil following decision,17422
4,electoral court tse,17422
5,following decision of,17422
6,was blocked in,17422
7,of the superior,17422
8,this was blocked,17422
9,in brazil following,17422


### 22. As 30 mensagens mais positivas (distintas)

In [54]:
top_positive_messages = conn.execute("""
  SELECT DISTINCT text_content_anonymous, score_sentiment
  FROM messages
  WHERE text_content_anonymous IS NOT NULL
    AND text_content_anonymous != ''
    AND score_sentiment IS NOT NULL AND score_sentiment >= 0.05
  ORDER BY score_sentiment DESC
  LIMIT 30;
""").df()
top_positive_messages

,text_content_anonymous,score_sentiment
0,"Shemoneh Esreh – Amidá – A Grande Oração\n\n ,...",0.9992
1,"MENTIROSA!!!\n\n\nPensa direita, não é correto...",0.9990
2,"Pensa direita, não é correto compartilhar um v...",0.9990
3,"בונה ירושלים, Bo'ne Yetrushalayim - Benção da ...",0.9989
4,Os Sete Samurais \n \nQuando falamos sobre For...,0.9988
5,MANIFESTO À NAÇÃO BRASILEIRA\n\nEM DEFESA DO B...,0.9987
6,"Olá ,\n\nUM AMIGO ME PERGUNTOU EM QUEM EU VOTE...",0.9986
7,BOM DIA A TODOS. ENTENDO QUE ESTAMOS VIVENDO D...,0.9986
8,\nA Viva Esperança\n\nDepois das saudações in...,0.9985
9,Uma forma bem legal\nPara quem ainda não sabe ...,0.9985


### 23. As 30 mensagens mais negativas (distintas)

In [55]:
top_negative_messages = conn.execute("""
  SELECT DISTINCT text_content_anonymous, score_sentiment
  FROM messages
  WHERE text_content_anonymous IS NOT NULL
    AND text_content_anonymous != ''
    AND score_sentiment IS NOT NULL AND score_sentiment <= 0.05
  ORDER BY score_sentiment ASC
  LIMIT 30;
""").df()
top_negative_messages

,text_content_anonymous,score_sentiment
0,CAPITÃOZINHO DE MERDA 💩\nCAPITÃOZINHO DE MERDA...,-1.0000
1,"E nós aqui nos sacrificando, aguentando sol, c...",-0.9999
2,Intervenção é meu pau na tua mão\nIntervenção ...,-0.9999
3,Intervenção é meu pau na tua mão\nIntervenção ...,-0.9999
4,ai ta na rua da um joinha 👍Quem ai ta na rua d...,-0.9999
5,ME CÚ NO PAU DO BOLSONARO \nME CÚ NO PAU DO BO...,-0.9999
6,"Gente, a Zambelli nos abandonou! Traiu a nossa...",-0.9999
7,Quem ai ta na rua da um joinha 👍Quem ai ta na ...,-0.9998
8,nha 👍Quem ai ta na rua da um joinha 👍Quem ai t...,-0.9998
9,a rua da um joinha 👍Quem ai ta na rua da um jo...,-0.9998


### 24. O usuário mais otimista

In [56]:
top_positive_user = conn.execute("""
SELECT
    id_member_anonymous,
    AVG(score_sentiment) AS avg_sentiment,
    COUNT(*) AS total_messages
FROM messages
WHERE score_sentiment IS NOT NULL
  AND id_member_anonymous IS NOT NULL
GROUP BY id_member_anonymous
ORDER BY avg_sentiment DESC
LIMIT 1;
""").df()
top_positive_user

,id_member_anonymous,avg_sentiment,total_messages
0,016723c9eb650754d3dfe9b3ba69e5c2,0.9953,1


In [57]:
top_positive_user_10 = conn.execute("""
SELECT
    id_member_anonymous,
    AVG(score_sentiment) AS avg_sentiment,
    COUNT(*) AS total_messages
FROM messages
WHERE score_sentiment IS NOT NULL
  AND id_member_anonymous IS NOT NULL
GROUP BY id_member_anonymous
HAVING COUNT(*) >= 10
ORDER BY avg_sentiment DESC
LIMIT 1;
""").df()
top_positive_user_10

,id_member_anonymous,avg_sentiment,total_messages
0,bab79a5f7c599d5b5b9155d5d3d8b570,0.92,25


### 25. O usuário mais pessimista

In [58]:
top_negative_user = conn.execute("""
SELECT
    id_member_anonymous,
    AVG(score_sentiment) AS avg_sentiment,
    COUNT(*) AS total_messages
FROM messages
WHERE score_sentiment IS NOT NULL
  AND id_member_anonymous IS NOT NULL
GROUP BY id_member_anonymous
ORDER BY avg_sentiment ASC
LIMIT 1;
""").df()
top_negative_user

,id_member_anonymous,avg_sentiment,total_messages
0,243a43d802953032e1ea9a363546c3b8,-0.9992,1


In [59]:
top_negative_user_10 = conn.execute("""
SELECT
    id_member_anonymous,
    AVG(score_sentiment) AS avg_sentiment,
    COUNT(*) AS total_messages
FROM messages
WHERE score_sentiment IS NOT NULL
  AND id_member_anonymous IS NOT NULL
GROUP BY id_member_anonymous
HAVING COUNT(*) >= 10
ORDER BY avg_sentiment ASC
LIMIT 1;
""").df()
top_negative_user_10

,id_member_anonymous,avg_sentiment,total_messages
0,9062da565a741b1862110f1a6110b937,-0.9325,15


### 26. As 30 maiores mensagens

In [60]:
top_longest_messages = conn.execute("""
SELECT
    id_message,
    id_member_anonymous,
    LENGTH(text_content_anonymous) AS message_length,
    text_content_anonymous
FROM messages
WHERE text_content_anonymous IS NOT NULL
  AND text_content_anonymous != ''
ORDER BY message_length DESC
LIMIT 30;
""").df()
top_longest_messages

,id_message,id_member_anonymous,message_length,text_content_anonymous
0,523456,fea1db96601338fc9ed2b2b190c7574f,4096,Trecho do livro “Prática da Conspiração A Agen...
1,165582,fea1db96601338fc9ed2b2b190c7574f,4096,Trecho do livro “Prática da Conspiração A Agen...
2,20207,adc36f63f2f969861fd6dfaea80f7b91,4096,"1: Sansão foi a Gaza, e viu ali uma prostituta..."
3,20282,adc36f63f2f969861fd6dfaea80f7b91,4096,"1: Aconteceu também naqueles dias, quando não ..."
4,20335,adc36f63f2f969861fd6dfaea80f7b91,4096,1: Naqueles dias não havia melech em Yisra-EL;...
5,20383,adc36f63f2f969861fd6dfaea80f7b91,4096,"1: Aconteceu também naqueles dias, quando não ..."
6,19479,None,4096,COMO O SOCIALISMO DESTRUIU A VENEZUELA EM POUC...
7,9088,f4561a21fb26cfbebf48dcd5c557f82f,4096,PROPOSTAS DE BOLSONARO PARA 2023 - POSTADAS AG...
8,1007746,bd193b0dc70113a2d7032045177920e6,4096,[Encaminhado de FYI-Brasil (Sú)]\n🚨ELEIÇÕES DE...
9,19794,08c5c519d8d1ec8b2033ac8e2b74b510,4096,ROIA♘ ♞HERON★�CAVALO★§�DE★§�§TROIA♘ ♞HERON★�CA...


### 27. As 30 menores mensagens

In [61]:
top_shortest_messages = conn.execute("""
SELECT
    id_message,
    id_member_anonymous,
    LENGTH(text_content_anonymous) AS message_length,
    text_content_anonymous
FROM messages
WHERE text_content_anonymous IS NOT NULL
  AND text_content_anonymous != ''
ORDER BY message_length ASC
LIMIT 30;
""").df()
top_shortest_messages

,id_message,id_member_anonymous,message_length,text_content_anonymous
0,10086,None,1,😂
1,1169527,None,1,🎯
2,48089,79fa510c3ab44548d903284ab9162c9d,1,😁
3,1143211,None,1,🎯
4,523527,b2022827bfa661d21fa9723cbe36f292,1,🤔
5,1143464,01642d1a0ebc95e3be57710a9e077dd7,1,😏
6,1057580,4f7d493f0f6222d56e5b19a4f7c336cc,1,🤣
7,448840,None,1,😅
8,449381,None,1,🎯
9,1061049,None,1,👇


### 28. O dia em que foi publicado a maior quantidade de mensagens

In [62]:
day_with_most_messages = conn.execute("""
SELECT
    CAST(date_message AS DATE) AS message_day,
    COUNT(*) AS total_messages
FROM messages
WHERE date_message IS NOT NULL
GROUP BY message_day
ORDER BY total_messages DESC
LIMIT 1;
""").df()
day_with_most_messages

,message_day,total_messages
0,2022-10-02,25192


### 29. As mensagens que possuem as palavras “FACÇÃO” e “CRIMINOSA”;

In [63]:
messages_with_faction_and_criminal = conn.execute("""
    SELECT
        id_message,
        id_member_anonymous,
        text_content_anonymous
    FROM messages
    WHERE text_content_anonymous IS NOT NULL
      AND LOWER(text_content_anonymous) LIKE '%facção%'
      AND LOWER(text_content_anonymous) LIKE '%criminosa%'
""").df()
messages_with_faction_and_criminal

,id_message,id_member_anonymous,text_content_anonymous
0,19285,None,"Pensem!!!!!!'Se Houve fraude ,Nada foi feito ..."
1,22872,None,#lulaladrãoseulugarénaprisão \n#PTFACÇÃOCRIMIN...
2,105417,530db5d239f941ef44aae189f20b44a2,"Se depois de ler esse resumo do Plano do PT, v..."
3,1090985,530db5d239f941ef44aae189f20b44a2,"Se depois de ler esse resumo do Plano do PT, v..."
4,1097866,None,É exatamente isso que a facção criminosa CHAM...
...,...,...,...
134,1210244,None,😣🤯🤐 Alexandre de Moraes (Secretário-Geral do P...
135,973541,None,😣🤯🤐 Alexandre de Moraes (Secretário-Geral do P...
136,19115,None,😣🤯🤐 Alexandre de Moraes (Secretário-Geral do P...
137,53201,76b7d0b3c652aaefb984081e503af0eb,Agressões covardes é termos uma facção crimino...


### 30. As mensagens que possuem a palavra “SEGURANÇA”

In [64]:
messages_with_seguranca = conn.execute("""
    SELECT
        id_message,
        id_member_anonymous,
        text_content_anonymous
    FROM messages
    WHERE text_content_anonymous IS NOT NULL
      AND LOWER(text_content_anonymous) LIKE '%segurança%'
""").df()
messages_with_seguranca

,id_message,id_member_anonymous,text_content_anonymous
0,8766,None,🌳 Mato Grosso intensifica ações contra crimes ...
1,314893,b4ce87d44421ef3d2c4710051df5fe40,⚡️🇧🇷 Actus Liberandi - Ordem De Cristo 🇧🇷\n\n✨...
2,314913,b4ce87d44421ef3d2c4710051df5fe40,⚡️🇧🇷 Actus Liberandi - Ordem De Cristo 🇧🇷\n\n✨...
3,16491,3ec4f5ab6a5754fc2f67d29fa203ff56,No Telegram entrem em Configurações > Privacid...
4,16492,3ec4f5ab6a5754fc2f67d29fa203ff56,No Telegram entrem em Configurações > Privacid...
...,...,...,...
2788,188195,None,Profissão médica instada a dizer às mulheres a...
2789,1285805,None,"TSE, o relatório das Forças Armadas e o caso M..."
2790,121386,None,"TSE, o relatório das Forças Armadas e o caso M..."
2791,9300,None,✅ Notícias do Dia - Ano 6 n° 2183\nDia 11 de ...
